In [159]:
from collections import deque
import time
import math
from datetime import datetime, timedelta
import pandas as pd

In [160]:
class Visitor: # generate unique Visitor
    def __init__(self, id, itinerary, fast_pass: int, arrival_time, current_action):
        """
        :param id: a unique ID
        :param itinerary: a list of itinerary (where they will go, in sequence)
        :param fast_pass: 0 for regular ticket, 1 for fast-pass ticket
        """
        self.id = id
        self.fast_pass = fast_pass 
        self.itinerary = itinerary 
        self.current_time = arrival_time # First set to when they are 'spawned'
        self.accumulated_waiting_time = 0 ### LINK TO JON's part
        self.accumulated_satisfactory_score = 0 ### LINK TO JON's part
        
        self.current_action = current_action
        self.next_action = None
        self.current_ride_start = None # The moment they get served from the queue
        
        self.service_start_time = None
        self.service_end_time = None

    def __repr__(self): # representation method 
        return f"Visitor {self.id, self.fast_pass}"
    
    def choose_action(self):
        if self.itinerary:
            self.next_action = self.itinerary.pop(0)
        print(f"{self} is heading to {self.next_action}.")
        return self.next_action
    
    def travel(self, distance): # visitor travels between nodes ### link to Jon's
        """
        :param distance: distance between two position
        """
        time_travelled = distance # It takes 1 minute to travel distance = 1
        self.current_time += timedelta(minutes=int(time_travelled))


In [161]:
class Attraction: # Including shows, rides
    def __init__(self, name: str, ride_duration: int, ride_capacity: int):
        """
        Initialize an attraction with normal and express queues.
        :param name: Name of the attraction.
        :param ride_duration: Duration of the ride
        :param ride_capacity: Number of visitors that can be processed at once.
        """
        self.name = name
        self.holding_visitors = deque() # To store visitors before adding them to regular/fast-pass queue
        self.regular_queue = deque()  
        self.fast_pass_queue = deque()
        self.ride_duration = int(ride_duration)  # Time of each ride
        self.ride_capacity = ride_capacity # Max number of ppl a ride can take
        self.current_time = 10  # Simulation time in seconds
        self.total_served = 0  # cumulative number of served visitors 
        self.rides_served = 0 # Cumulative number of rides served in the current simulation, Flag for early termination
    
    def get_rides_served(self):
        return self.rides_served

    def add_visitor(self, visitor):
        """
        :param visitor: a Visitor object
        """
        if isinstance(visitor, Visitor):
            if visitor.current_time <= self.current_time:
                if visitor.fast_pass == 1:
                    self.fast_pass_queue.append(visitor)
                    print(f"{visitor} added to the fast-pass queue at time {self.current_time}.") # for debugtest
                else:
                    self.regular_queue.append(visitor)
                    print(f"{visitor} added to the regular queue at time {self.current_time}.") # for debugtest
            else:
                print(f"{visitor} has not yet arrived at time {self.current_time}.")
        else:
            print("Only Visitor objects can be added to the queue.") # for debugtest

    def process_queue(self):
        # assumption: ride_capacity is EVEN number; fast-pass vs regular queue each take up 1/2 of ride_capacity
        # operation: take min(1/2*ride_capacity visitors out of fast_pass_queue, len(fast_pass_queue)), the rest from regular_queue
        
        riding_visitors = [] # list of visitors being served in the current ride
        
        # take min(1/2*ride_capacity visitors out of fast_pass_queue, len(fast_pass_queue))
        if self.fast_pass_queue: # while fast_pass_queue has visitors
            for _ in range(min(self.ride_capacity//2, len(self.fast_pass_queue))):
                person = self.fast_pass_queue.popleft()
                if person not in riding_visitors:
                    riding_visitors.append(person) # Get visitors in fast_pass_queue
                    print(f"{person} from Fast Pass Queue is being served.") # for debugging
                    self.total_served += 1
                    print("total serving so far:", self.total_served) # for debugging
        
        fast_pass_filled_ride = len(riding_visitors) # number of fast_pass visitors going to the current ride
        
        # the rest from regular_queue
        if self.regular_queue:
            for _ in range(min(self.ride_capacity - fast_pass_filled_ride, len(self.regular_queue))):
                person = self.regular_queue.popleft()
                if person not in riding_visitors:
                    riding_visitors.append(person) # Get visitors in regular_queue
                    print(f"{person} from Regular Queue is being served.") # for debugging
                    self.total_served += 1
                    print("total serving so far:", self.total_served) # for debugging
        
        if not self.fast_pass_queue and not self.regular_queue:
            print("No one is waiting in the queues at time", self.current_time)
        
        self.rides_served += 1
        print(f"Number of rides so far is{self.rides_served}") # debugging

        # The time now is right before the ride starts. We can update visitor's current time to calculate waiting time
        for visitor in riding_visitors:
            visitor.current_ride_start = self.current_time
            wait_time = visitor.current_ride_start - visitor.current_time # wait time = time the ride start - time visitor arrive at the queue
            print(f"{visitor} waited for {wait_time}") # debugging
            visitor.accumulated_waiting_time += wait_time
            # Then, update the time for Visitor objects 
            visitor.current_time += timedelta(minutes=0 if math.isnan(self.ride_duration) else self.ride_duration)
            # and we send them off to the next item on their itinerary:
            visitor.current_action = visitor.next_action
            visitor.next_action = visitor.choose_action()

        
        # Lastly, we update the time of this Attraction object:
        self.current_time += timedelta(minutes=0 if math.isnan(self.ride_duration) else self.ride_duration)
        print(f"The time at {self.name} now is {self.current_time}.")  

    def get_data(self): # Note: should only be called AFTER calling process_queue()
        if math.isnan(self.ride_duration):
            self.ride_duration = 0
        crowd_level = {'fast_pass_queue':len(self.fast_pass_queue), 
                       'regular_queue':len(self.regular_queue)}
        curr_wait_time = {'fast_pass_queue':math.floor(crowd_level['fast_pass_queue']/self.ride_capacity)*self.ride_duration, 
                          'regular_queue':math.floor(crowd_level['regular_queue']/self.ride_capacity)*self.ride_duration}
        return self.current_time, crowd_level, curr_wait_time
    
    

In [162]:
class Seasonal: # Including shows, rides, has fixed timing
    def __init__(self, name: str, ride_duration: int, ride_capacity: int, timeslot):
        """
        Initialize an attraction with normal and express queues.
        :param name: Name of the attraction.
        :param ride_duration: Duration of the ride
        :param ride_capacity: Number of visitors that can be processed at once.
        :param timeslot: schedule of the seasonal attraction, list
        """
        self.name = name
        self.holding_visitors = deque() # To store visitors before adding them to regular/fast-pass queue
        self.regular_queue = deque()  
        self.fast_pass_queue = deque()
        self.ride_duration = int(ride_duration)  # Time of each ride
        self.ride_capacity = ride_capacity # Max number of ppl a ride can take
        self.timeslot = timeslot
        self.current_time = timeslot.pop(0) 
        self.total_served = 0  # cumulative number of served visitors 
        self.rides_served = 0 # Cumulative number of rides served in the current simulation, Flag for early termination
    
    def get_rides_served(self):
        return self.rides_served

    def add_visitor(self, visitor):
        """
        :param visitor: a Visitor object
        """
        if isinstance(visitor, Visitor):
            if visitor.arrival_time <= self.current_time:
                if visitor.fast_pass == 1:
                    self.fast_pass_queue.append(visitor)
                    print(f"{visitor} added to the fast-pass queue at time {self.current_time}.") # for debugtest
                else:
                    self.regular_queue.append(visitor)
                    print(f"{visitor} added to the regular queue at time {self.current_time}.") # for debugtest
            else:
                print(f"{visitor} has not yet arrived at time {self.current_time}.")
        else:
            print("Only Visitor objects can be added to the queue.") # for debugtest

    def process_queue(self):
        # assumption: ride_capacity is EVEN number; fast-pass vs regular queue each take up 1/2 of ride_capacity
        # operation: take min(1/2*ride_capacity visitors out of fast_pass_queue, len(fast_pass_queue)), the rest from regular_queue
        
        riding_visitors = [] # list of visitors being served in the current ride
        
        # take min(1/2*ride_capacity visitors out of fast_pass_queue, len(fast_pass_queue))
        if self.fast_pass_queue: # while fast_pass_queue has visitors
            for _ in range(min(self.ride_capacity//2, len(self.fast_pass_queue))):
                person = self.fast_pass_queue.popleft()
                if person not in riding_visitors:
                    riding_visitors.append(person) # Get visitors in fast_pass_queue
                    print(f"{person} from Fast Pass Queue is being served.") # for debugging
                    self.total_served += 1
                    print("total serving so far:", self.total_served) # for debugging
        
        fast_pass_filled_ride = len(riding_visitors) # number of fast_pass visitors going to the current ride
        
        # the rest from regular_queue
        if self.regular_queue:
            for _ in range(min(self.ride_capacity - fast_pass_filled_ride, len(self.regular_queue))):
                person = self.regular_queue.popleft()
                if person not in riding_visitors:
                    riding_visitors.append(person) # Get visitors in regular_queue
                    print(f"{person} from Regular Queue is being served.") # for debugging
                    self.total_served += 1
                    print("total serving so far:", self.total_served) # for debugging
        
        if not self.fast_pass_queue and not self.regular_queue:
            print("No one is waiting in the queues at time", self.current_time)
        
        self.rides_served += 1
        print(f"Number of rides so far is{self.rides_served}") # debugging

        # The time now is right before the ride starts. We can update visitor's current time to calculate waiting time
        for visitor in riding_visitors:
            visitor.current_ride_start = self.current_time
            wait_time = visitor.current_ride_start - visitor.current_time # wait time = time the ride start - time visitor arrive at the queue
            print(f"{visitor} waited for {wait_time}") # debugging
            visitor.accumulated_waiting_time += wait_time
            # Then, update the time for Visitor objects 
            visitor.current_time += timedelta(minutes=0 if math.isnan(self.ride_duration) else self.ride_duration) 
            # and we send them off to the next item on their itinerary:
            visitor.current_action = visitor.next_action
            visitor.next_action = visitor.choose_action()

        
        # Lastly, we update the time of this Attraction object:
        self.current_time =self.timeslot.pop(0)
        print(f"The time at {self.name} now is {self.current_time}.")  

    def get_data(self): # Note: should only be called AFTER calling process_queue()
        if math.isnan(self.ride_duration):
            self.ride_duration = 0
        crowd_level = {'fast_pass_queue':len(self.fast_pass_queue), 
                       'regular_queue':len(self.regular_queue)}
        curr_wait_time = {'fast_pass_queue':math.floor(crowd_level['fast_pass_queue']/self.ride_capacity)*self.ride_duration, 
                          'regular_queue':math.floor(crowd_level['regular_queue']/self.ride_capacity)*self.ride_duration}
        return self.current_time, crowd_level, curr_wait_time
    

In [163]:
class Utility: # Including toilets, dining outlets, souvenir shops
    def __init__(self, name: str, service_duration: int, util_capacity: int):
        """
        Initialize a utility in the part.
        :param name: Name of the utility.
        :param service_time: Duration spent inside the Utility
        :param util_capacity: Number of visitors that can be processed at once.
        """
        self.name = name
        self.service_duration = int(service_duration)
        self.util_capacity = util_capacity 
        self.holding_visitors = deque() # To store coming visitors before processing them
        self.queue = deque()
        self.serving_visitors = deque() # To track visitors being served at the current moment
        self.current_time = None
        self.total_served = 0  # cumulative number of served visitors 

    def add_visitor(self, visitor):
        """
        :param visitor: a Visitor object
        """
        if isinstance(visitor, Visitor):
            if visitor.current_time <= self.current_time:
                self.queue.append(visitor)
                print(f"{visitor} added to the queue at time {self.current_time}.") # for debugtest
            else:
                print(f"{visitor} has not yet arrived at time {self.current_time}.") # for debugtest
        else:
            print("Only Visitor objects can be added to the queue.") # for debugtest


    def process_queue(self):
        """
        Process the queue continuously, allowing visitors to enter as soon as space is available.
        """
        # Check if any serving visitors have completed their service duration and can leave
        self.update_serving_visitors()

        # Move visitors from the queue to service as capacity allows
        while len(self.serving_visitors) < self.util_capacity and self.queue:
            visitor = self.queue.popleft()
            visitor.service_start_time = self.current_time  # Set the start time for service
            visitor.service_end_time = self.current_time + timedelta(minutes=0 if math.isnan(self.ride_duration) else self.ride_duration)  # Set end time
            visitor.waiting_time = (visitor.start_time - visitor.arrival_time).total_seconds() // 60  # In minutes
            self.serving_visitors.append(visitor)
            self.total_served += 1
            print(f"{visitor} is now being served. Waiting time was {visitor.waiting_time} minutes.")

    def update_serving_visitors(self):
        """
        Removes visitors from `serving_visitors` who have completed their service.
        """
        while self.serving_visitors and self.serving_visitors[0].service_end_time <= self.current_time:
            completed_visitor = self.serving_visitors.popleft()
            # and we send them off to the next item on their itinerary:
            completed_visitor.current_action = completed_visitor.next_action
            completed_visitor.next_action = completed_visitor.choose_action()
            print(f"{completed_visitor} has completed their time in {self.name} and is leaving.")

    def advance_time(self, minutes: int):
        """
        Advances the current time by a specified number of minutes.
        """
        self.current_time += timedelta(minutes=minutes)
        print(f"The time now is {self.current_time}.")

    def get_data(self):
        """
        Returns the current crowd level in the queue and 
        returns the estimated wait time based on the current crowd level and service duration.
        """
        if math.isnan(self.service_duration):
            self.service_duration = 0
        crowd_level = len(self.queue)
        print(f"At {self.current_time}, the crowd level is {crowd_level}")
        curr_wait_time = math.ceil(crowd_level / self.util_capacity) * self.service_duration
        print(f"At {self.current_time}, the estimated wait time is {curr_wait_time} minutes")
        return self.current_time, crowd_level, curr_wait_time
    
    def get_visitors_served(self):
        return self.total_served


In [164]:
import random

class ThemePark:
    def __init__(self, spawning_dict, all_possible_itineraries):
        """
        :param spawning_dict: a dictionary containing number of new visitors for each time slot

        """
        self.spawning_dict = spawning_dict
        self.all_possible_itineraries = all_possible_itineraries
        self.global_time = None  # Overall time counter for the simulation, take from the time_slot object
        self.attractions = []
        self.utilities = []
        self.seasonals = []
        self.existing_visitors = {} # A dictionary to store the lists of spawning visitors at each time increments
        self.visitors_count = 0 # to track and create new unique visitors

    def add_attraction(self, attraction: Attraction):
        """
        Add an attraction to the park.
        :param attraction: Attraction object.
        """
        self.attractions.append(attraction)

    def add_utility(self, utility: Utility):
        """
        Add a utility to the park.
        :param utility: Utility object.
        """
        self.utilities.append(utility)

    def add_seasonal(self, seasonal: Seasonal):
        """
        Add a seasonal attraction to the park.
        :param seasonal: Seasonal object.
        """
        self.seasonals.append(seasonal)

    def spawn_visitor(self, time):
        """ 
        :param time: time slot
        """
        visitors = [] # List to store Visitor objects
        for _ in range(self.spawning_dict[time.strftime('%H:%M')]):
            new_visitor = Visitor(id= self.visitors_count, 
                                    itinerary=random.choice(self.all_possible_itineraries), 
                                    fast_pass=random.choices([0,1], weights=[0.8, 0.2],k=1), # 20% of purchasing express pass
                                    arrival_time=time,
                                    current_action='Entrance')
            self.visitors_count += 1
            visitors.append(new_visitor)
        return visitors
    
    def open_park(self, time): 
        """ 
        :param time: 
        """
        for item in self.attractions+self.utilities+self.seasonals:
            item.current_time = time
        # for utility in :
        #     utility.current_time = time
        # for seasonal in :
        #     seasonal

    def advance_time(self, minutes: int):
        """
        Advances the current time by a specified number of minutes.
        """
        self.global_time += timedelta(minutes=minutes)
        print(f"The time now is {self.current_time}.")



From here on is the setting up of the simulation.

In [165]:
#STEP 1: SPAWNING VISITORS, MAKE visitor_generator_df

spawning_df = pd.read_csv('../data/spawning.csv')

# Convert columns to the desired types
spawning_df['Average Wait Time'] = spawning_df['Average Wait Time'].astype(float)  # Convert to float
spawning_df['Number of Visitors'] = spawning_df['Number of Visitors'].astype(int)  # Convert to int
spawning_df['Number of New Visitors Arriving'] = spawning_df['Number of New Visitors Arriving'].astype(int)  # Convert to int

# print(spawning_df) # debugging

# Convert df into dictionary for easier reference
spawning = {}
for index, row in spawning_df.iterrows():
    number = row['Number of New Visitors Arriving']
    if number >= 0:
        spawning[row['Time']] = number
    else:
        spawning[row['Time']] = 0

print(spawning) # debugging

# take reference from Subgroup A for demand forecasting

{'10:00': 35, '10:05': 0, '10:10': 0, '10:15': 50, '10:20': 0, '10:25': 0, '10:30': 50, '10:35': 50, '10:40': 0, '10:45': 50, '10:50': 0, '10:55': 100, '11:00': 50, '11:05': 0, '11:10': 0, '11:15': 50, '11:20': 50, '11:25': 0, '11:30': 0, '11:35': 50, '11:40': 0, '11:45': 50, '11:50': 100, '11:55': 0, '12:00': 0, '12:05': 150, '12:10': 0, '12:15': 50, '12:20': 0, '12:25': 0, '12:30': 50, '12:35': 0, '12:40': 0, '12:45': 50, '12:50': 0, '12:55': 50, '13:00': 0, '13:05': 0, '13:10': 50, '13:15': 0, '13:20': 0, '13:25': 0, '13:30': 100, '13:35': 0, '13:40': 100, '13:45': 0, '13:50': 50, '13:55': 0, '14:00': 0, '14:05': 0, '14:10': 50, '14:15': 50, '14:20': 0, '14:25': 0, '14:30': 0, '14:35': 0, '14:40': 50, '14:45': 0, '14:50': 0, '14:55': 0, '15:00': 50, '15:05': 0, '15:10': 50, '15:15': 0, '15:20': 0, '15:25': 0, '15:30': 0, '15:35': 0, '15:40': 0, '15:45': 0, '15:50': 50, '15:55': 0, '16:00': 0, '16:05': 0, '16:10': 0, '16:15': 0, '16:20': 0, '16:25': 50, '16:30': 0, '16:35': 0, '16:40

In [166]:

itineraries = pd.read_csv('../data/itinerary_no_H.csv')
itineraries['path list'] = itineraries['path'].str.split()
print(itineraries) # debugging

# Flatten all lists into a single list
all_possible_itineraries = [sublist for sublist in itineraries['path list']]

print(type(all_possible_itineraries[1])) # debugging, each item in the flattened list is still a list

                  path                    path list
0      E A D B D E I C     [E, A, D, B, D, E, I, C]
1    J G I E B C A K D  [J, G, I, E, B, C, A, K, D]
2         E K J I E B            [E, K, J, I, E, B]
3    A B C E I G K J D  [A, B, C, E, I, G, K, J, D]
4    A E I C J D K G B  [A, E, I, C, J, D, K, G, B]
..                 ...                          ...
150    J A I C E B G D     [J, A, I, C, E, B, G, D]
151    J A I C E B G D     [J, A, I, C, E, B, G, D]
152    A I C J E B G D     [A, I, C, J, E, B, G, D]
153    A I J C E B G D     [A, I, J, C, E, B, G, D]
154    I A B J E G C D     [I, A, B, J, E, G, C, D]

[155 rows x 2 columns]
<class 'list'>


In [167]:
#STEP 2: CREATING NODES, MAKE NODE LIST 

# Create Attraction objects from csv file
# Columns needed: name: str, ride_duration: int, ride_capacity: int

attraction_generator_df = pd.read_csv('../data/theme_park_nodes.csv')

attractions = [] # List to store Attraction objects
utilities = []
seasonals = []

for index, row in attraction_generator_df.iterrows():
    if row['type'] == 'Ride':
        new_attraction = Attraction(name=row['name'], ride_duration=row['duration'], ride_capacity=row['capacity']) # change according to the finalised csv
        attractions.append(new_attraction) 
    if row['type'] == 'Seasonal':

        time_string = row['timeslots']
        time_list = time_string.split(", ") 
        datetime_list = [datetime.strptime(item, "%H:%M") for item in time_list]


        print(datetime_list)
        new_seasonal = Seasonal(name=row['name'], ride_duration=row['duration'], ride_capacity=row['capacity'], timeslot=datetime_list) # change according to the finalised csv
        seasonals.append(new_seasonal)     
    if row['type'] in ['Restroom', 'Dining Outlet', 'Food Cart', 'Retail']:
        new_utility = Utility(name=row['name'], service_duration=row['duration'], util_capacity=row['capacity'])
        utilities.append(new_utility)

print(attractions) # debugging
print(utilities) # debugging
print(seasonals) # debugging

print(attraction_generator_df[attraction_generator_df['index'] == 'A']['name']) # debugging
print(type(attraction_generator_df[attraction_generator_df['index'] == 'A']['name'])) # debugging


ValueError: cannot convert float NaN to integer

In [ ]:
#STEP 3: CREATING USS, PUT THINGS TOGETHER

# Creating ThemePark object
USS = ThemePark(spawning, all_possible_itineraries)

# Add Attraction objects to the park
for item in attractions:
    USS.add_attraction(item)

# Add Utility objects to the park
for item in utilities:
    USS.add_utility(item)

# Add Seasonal objects to the park
for item in seasonals:
    USS.add_seasonal(item)

# Open the park at 10:00, update time of all attractions and utilities
USS.global_time = datetime.strptime("10:00", "%H:%M")
USS.open_park(USS.global_time)

# Create a dictionary to store data of waiting time and crowd level across attractions and utilities across the theme park
park_data = {}

In [ ]:
#STEP 4: IMPORTING EDGES (to facilitate Visitors travelling from one node to another)
paths_df = pd.read_csv('../data/theme_park_paths.csv')

print(paths_df) # debugging

           source                       target  \
0     Accelerator                  Accelerator   
1     Accelerator  Battlestar Galactica: CYLON   
2     Accelerator  Battlestar Galactica: HUMAN   
3     Accelerator                 Canopy Flyer   
4     Accelerator                 Dino Soarin'   
...           ...                          ...   
3595     Entrance                   Restroom 4   
3596     Entrance                   Restroom 5   
3597     Entrance                   Restroom 6   
3598     Entrance                   Restroom 7   
3599     Entrance                     Entrance   

                                                  nodes  distance  
0                                       ['Accelerator']         0  
1        ['Accelerator', 'Battlestar Galactica: CYLON']         4  
2     ['Accelerator', 'Battlestar Galactica: CYLON',...         5  
3     ['Accelerator', 'Restroom 3', 'Galactic Treats...        30  
4     ['Accelerator', 'Restroom 3', 'Galactic Treats...    

In [ ]:
#STEP 5: UP AND RUNNING

while USS.global_time <= datetime.strptime("12:00", "%H:%M"): # testing for 2 hours

    # Create Visitors objects
    USS.existing_visitors[USS.global_time] = USS.spawn_visitor(USS.global_time) 
    
    # Process the Visitor objects: passing them to the corresponding node according to their itinerary

    # Choose the next node based on their itinerary:
    for person in USS.existing_visitors[USS.global_time]:
        choice_index = person.choose_action()
        choices = attraction_generator_df[attraction_generator_df['index'] == choice_index]['name']
        if not choices.tolist():
            print(choice_index)
        if person.itinerary:
            person.next_action = random.choices(choices.tolist(),k=1)[0] # because random.choices() return a list, therefore need index
    
    # Passing them to the corresponding node:
    for item in USS.utilities+USS.attractions+USS.seasonals:
        for person in USS.existing_visitors[USS.global_time]:
            if person.next_action == item.name:
                # Get the single value if there's only one match
                # Filter paths_df to find the matching rows
                matching_rows = paths_df[(paths_df['source'] == person.current_action) & (paths_df['target'] == person.next_action)]

                # Check if the filtered DataFrame is empty
                if not matching_rows.empty:
                    # Access the 'distance' value from the first matching row
                    distance = matching_rows['distance'].iloc[0]
                else:
                    # Handle the case where no match is found
                    print('Error: No matching path found for source',person.current_action ,'and target', person.next_action)
                    distance = None  # or some default value

                distance = paths_df[(paths_df['source']==person.current_action) & (paths_df['target']==person.next_action)]['distance'].iloc[0] # should be from a unique entry
                person.travel(distance) # The visitor travel to that attraction/utility
                # note: cannot update person.current_action yet because still in iteration
                item.holding_visitors.append(person)
        
    # Running all attractions and utilities ONCE
    for item in USS.attractions:
        for visitor in item.holding_visitors:
            item.add_visitor(visitor)
        item.process_queue()
        time_slot, crowd, wait = item.get_data() # Note: should only be called AFTER calling process_queue()
        # Add the above info to park_data dictionary:
        park_data.setdefault(item.name, {}).setdefault(time_slot, {'crowd level': None, 'waiting time': None})
        park_data[item.name][time_slot]['crowd level'] = crowd
        park_data[item.name][time_slot]['waiting time'] = wait
        

    for item in USS.utilities:
        for visitor in item.holding_visitors:
            item.add_visitor(visitor)
        item.process_queue()
        time_slot, crowd, wait = item.get_data() 
        # Add the above info to park_data dictionary:
        park_data.setdefault(item.name, {}).setdefault(time_slot, {'crowd level': None, 'waiting time': None})
        park_data[item.name][time_slot]['crowd level'] = crowd
        park_data[item.name][time_slot]['waiting time'] = wait
        # Update current time for the utility
        item.advance_time(5)

    for item in USS.seasonals:
        for visitor in item.holding_visitors:
            item.add_visitor(visitor)
        item.process_queue()
        time_slot, crowd, wait = item.get_data() # Note: should only be called AFTER calling process_queue()
        # Add the above info to park_data dictionary:
        park_data.setdefault(item.name, {}).setdefault(time_slot, {'crowd level': None, 'waiting time': None})
        park_data[item.name][time_slot]['crowd level'] = crowd
        park_data[item.name][time_slot]['waiting time'] = wait
    
    # Update USS global time, ready for the next iteration:
    USS.advance_time(5)

# Upon finishing a time period, data on number of visitors served can be retrieved
quantity_data = {}

for item in USS.attractions:
    quantity_data[item.name] = item.total_served
    print(item, 'has served ', quantity_data[item.name], ' visitors.') # debugging

for item in USS.utilities:
    quantity_data[item.name] = item.get_visitors_served()   
    print(item, 'has served ', quantity_data[item.name], ' visitors.') # debugging

for item in USS.seasonals:
    quantity_data[item.name] = item.total_served
    print(item, 'has served ', quantity_data[item.name], ' visitors.') # debugging



print(park_data['Revenge of the Mummy']) # debugging


No one is waiting in the queues at time 1900-01-01 10:00:00
Number of rides so far is1
The time at Accelerator now is 1900-01-01 10:03:00.
Visitor (14, [1]) has not yet arrived at time 1900-01-01 10:00:00.
Visitor (15, [0]) has not yet arrived at time 1900-01-01 10:00:00.
Visitor (21, [0]) has not yet arrived at time 1900-01-01 10:00:00.
Visitor (30, [0]) has not yet arrived at time 1900-01-01 10:00:00.
No one is waiting in the queues at time 1900-01-01 10:00:00
Number of rides so far is1
The time at Battlestar Galactica: CYLON now is 1900-01-01 10:03:00.
Visitor (5, [0]) has not yet arrived at time 1900-01-01 10:00:00.
Visitor (16, [0]) has not yet arrived at time 1900-01-01 10:00:00.
Visitor (27, [0]) has not yet arrived at time 1900-01-01 10:00:00.
No one is waiting in the queues at time 1900-01-01 10:00:00
Number of rides so far is1
The time at Battlestar Galactica: HUMAN now is 1900-01-01 10:03:00.
Visitor (0, [0]) has not yet arrived at time 1900-01-01 10:00:00.
Visitor (4, [0]) 

ValueError: cannot convert float NaN to integer

In [ ]:
### Export relevant data into csv file for further analysis

# Dictionary to store DataFrames with dynamic names
dataframes = {}

for node, time_slot in park_data.items():
    for entry in time_slot.values():
        # Create a DataFrame
        df = pd.DataFrame(entry)
        
        # Create a dynamic key for the dictionary using the node name
        df_name = f'{node}_df'
        
        # Store the DataFrame in the dictionary
        dataframes[df_name] = df

# Example: Accessing the dynamically created DataFrames
print(dataframes['node1_df'])
print(dataframes['node2_df'])

for key, df in dataframes.items():    
    csv_file = '../data/simulation output/' + str(key) + '.csv'
    df.to_csv(csv_file, index=False)


In [168]:
print(park_data.items())


dict_items([('Accelerator', {datetime.datetime(1900, 1, 1, 10, 3): {'crowd level': {'fast_pass_queue': 0, 'regular_queue': 0}, 'waiting time': {'fast_pass_queue': 0.0, 'regular_queue': 0.0}}}), ('Battlestar Galactica: CYLON', {datetime.datetime(1900, 1, 1, 10, 3): {'crowd level': {'fast_pass_queue': 0, 'regular_queue': 0}, 'waiting time': {'fast_pass_queue': 0.0, 'regular_queue': 0.0}}}), ('Battlestar Galactica: HUMAN', {datetime.datetime(1900, 1, 1, 10, 3): {'crowd level': {'fast_pass_queue': 0, 'regular_queue': 0}, 'waiting time': {'fast_pass_queue': 0.0, 'regular_queue': 0.0}}}), ('Canopy Flyer', {datetime.datetime(1900, 1, 1, 10, 3): {'crowd level': {'fast_pass_queue': 0, 'regular_queue': 0}, 'waiting time': {'fast_pass_queue': 0.0, 'regular_queue': 0.0}}}), ("Dino Soarin'", {datetime.datetime(1900, 1, 1, 10, 3): {'crowd level': {'fast_pass_queue': 0, 'regular_queue': 0}, 'waiting time': {'fast_pass_queue': 0.0, 'regular_queue': 0.0}}})])
